In [1]:
!pip install mammoth
import mammoth
import re
from bs4 import BeautifulSoup


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# чтобы перевести из списка статей в формат словаря
def lines_to_entries(new_lines):
    entries = []
    ID = 1
    c = 0
    digits = ['I. ', 'II. ', 'III. ', 'IV. ', 'V. ', 'VI. ', 'VII. ', 'VIII. ', 'IX. ', 'X. ', 
              'I', 'II', 'III', 
              'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X',
              'I.', 'II.', 'III.', 'IV.', 'V.', 'VI.', 'VII.', 'VIII.', 'IX.', 'X.',
              'I ', 'II ', 'III ', 'IV ', 'V ', 'VI ', 'VII ', 'VIII ', 'IX ', 'X ']
    for line in new_lines:
      word = line.find('strong')
      new_entry = {}
      if word is not None:
        if word.text in digits:
            word = entries[c-1]['word']
            new_entry['id'] = ID
            new_entry['word'] = word
            new_entry['line'] = line
            new_entry['meanings'] = []
            entries.append(new_entry)            
        else:
            new_entry['id'] = ID
            new_entry['word'] = word.text.strip(' ').strip('I.')
            new_entry['line'] = line
            new_entry['meanings'] = []
            entries.append(new_entry)
        ID += 1
        c += 1
    return entries

In [3]:
def line_to_newmeaning(line):
      new_meaning = {}
      line = BeautifulSoup(line, 'html.parser')
      new_meaning['id'] = new_id
      new_meaning['comments'] = [i.text.strip(' ') for i in line.find_all('em') if i.text != ', ']

      if line.find_all('strong') != []:
          line.strong.extract()
      while line.find_all('em') != []:
          line.em.extract()


      examples_str = ''
      res = re.search(r'([а-я ,:;=\-\–\d\.()]+)([()АĂВИЙКЛԒЉМНЊӇОӨПРСТNУЎХШЩЫЄЭƏ\d\s ,:;=\-\–][аăвийклԓљмнњӈоөпрстnуўхшщыєэǝəӊъ\d ,:;\-\–]+.*[А-Я][а-я =;:,()ё\.\-\–]+)', line.text) 
      
      if res == None:
        res2 = re.search('([()АĂВИЙКЛԒЉМНЊӇОӨПРСТNУЎХШЩЫЄЭƏ\d\s ,:;\-\–][аăвийклԓљмнњӈоөпрстnуўхшщыєэǝəӊъ\d ,:;=\-\–]+)([А-Я][а-я =;:,()ё\.\-\–]+)', line.text)
        if res2 == None:    #только перевод
              translation = line.text.strip(' ').strip('.:;,')
              ex_orig = ''
              ex_trans = ''
        else: #только два предложения
            translation = ''
            examples_str = res2[0]

      else:   # перевод и два предложения 
          translation = res[1].strip(' ').strip('.;:,')
          examples_str = res[2]       
      new_meaning['translations'] = translation

      #примеры
      ex_pairs = examples_str.split(';')
      ex_pairs = [i for i in ex_pairs if i != ' ']
      examples = []
      for pair in ex_pairs:
        new_ex = {}
        p = re.search('([()АĂВИЙКЛԒЉМНЊӇОӨПРСТNУЎХШЩЫЄЭƏ\d\s ,:;\-\–][аăвийклԓљмнњӈоөпрстnуўхшщыєэǝəӊъ\d ,:;=\-\–]+)([А-Я][а-я =;:,()ё\.\-\–]+)', pair)
        if p != None:
          ex_orig = p[1]
          ex_trans = p[2]
          new_ex['example_original'] = ex_orig
          new_ex['example_translation'] = ex_trans
          examples.append(new_ex)
      new_meaning['examples'] = examples

      #референсы
      new_meaning['references'] = []
      references = {}
      references['entry_id'] = 0
      references['meaning_id'] = 0
      references['reference_comment'] = ''
      new_meaning['references'].append(references)
      
      

      return new_meaning

In [16]:
path = 'dict_Solovar_final.docx'
with open(path, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    html = result.value # The generated HTML
    messages = result.messages # Any messages, such as warnings during conversion
with open('Solovar.html', 'w', encoding = 'utf-8') as res:
    res.write(html)    
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
lines = soup.find_all('p')
new_lines = lines[134:6419]

In [20]:
entries = lines_to_entries(new_lines)

for entry in entries:
  # определяем, многозначное или нет
  l = entry['line']
  del entry['line']
  lines = re.split('\d+\.', str(l))
  lines = [i for i in lines if i != ''][1:]
  new_id = 0
  new_meaning = {}
  # тут для многозначных

  if lines != []:
    for line in  lines:
      new_id += 1
      new_meaning = line_to_newmeaning(line)
      entry['meanings'].append(new_meaning)
      new_meaning = {}
      
  # тут для однозначных
  else:
    new_id = 1
    new_meaning = line_to_newmeaning(str(l))
    entry['meanings'].append(new_meaning)

In [21]:
entries[:8]

[{'id': 1,
  'word': 'авəт',
  'meanings': [{'id': 1,
    'comments': ['сущ.'],
    'translations': 'высокий мыс',
    'examples': [],
    'references': [{'entry_id': 0,
      'meaning_id': 0,
      'reference_comment': ''}]}]},
 {'id': 2,
  'word': 'авка',
  'meanings': [{'id': 1,
    'comments': ['сущ.'],
    'translations': 'ручной олень',
    'examples': [],
    'references': [{'entry_id': 0,
      'meaning_id': 0,
      'reference_comment': ''}]}]},
 {'id': 3,
  'word': 'Авка йухан',
  'meanings': [{'id': 1,
    'comments': ['топоним.'],
    'translations': 'река Авка (ручного оленя)',
    'examples': [],
    'references': [{'entry_id': 0,
      'meaning_id': 0,
      'reference_comment': ''}]}]},
 {'id': 4,
  'word': 'авты',
  'meanings': [{'id': 1,
    'comments': ['гл.', 'суб.'],
    'translations': '  пахнуть',
    'examples': [{'example_original': 'Атǝм єпəԓ авəԓ ',
      'example_translation': 'Плохим запахом пахнет.'}],
    'references': [{'entry_id': 0,
      'meaning_id':

In [22]:
for i in entries:
  if i['word'] == 'ԓор':
    print(i)

{'id': 2331, 'word': 'ԓор', 'meanings': [{'id': 1, 'comments': [], 'translations': 'низкие места по берегам рек, на которых после спада воды  образуются озера', 'examples': [{'example_original': ' вөԓ ', 'example_translation': 'Посередине озера  один  большой  остров  есть'}], 'references': [{'entry_id': 0, 'meaning_id': 0, 'reference_comment': ''}]}, {'id': 2, 'comments': ['фолькл.'], 'translations': 'озерный', 'examples': [{'example_original': 'Ԓор хоԓəп ', 'example_translation': 'Озерная сеть'}, {'example_original': 'Ԓăхсəм ԓор вөԓтаԓ ', 'example_translation': 'Где-то озеро '}, {'example_original': ') ', 'example_translation': 'Семь озер видно (просека для перевеса).'}], 'references': [{'entry_id': 0, 'meaning_id': 0, 'reference_comment': ''}]}]}
